# **VIRTUAL RACE ENGINEER**

In [1]:
import fastf1 as ff
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import fastf1.plotting
fastf1.plotting.setup_mpl()
import fastf1.api as fap
fastf1.Cache.enable_cache(r"D:\Prabhu\SEM 7\F1 Data")
import re
import datetime as dt
import fastf1.mvapi as famp


d:\Prabhu\SEM 7\F1 Data\.venv\Lib\site-packages\fastf1\api.py:32: UserWarning: `fastf1.api` will be considered private in future releases and potentially be removed or changed!
  warnings.warn("`fastf1.api` will be considered private in future releases and "


### LOADING SESSION

In [2]:
session = ff.get_session(2023,'Dutch','R')
session.load()

core           INFO 	Loading data for Dutch Grand Prix - Race [v3.3.7]
req            INFO 	Using cached data for session_info
req            INFO 	Using cached data for driver_info
req            INFO 	Using cached data for session_status_data
req            INFO 	Using cached data for lap_count
req            INFO 	Using cached data for track_status_data
req            INFO 	Using cached data for _extended_timing_data
req            INFO 	Using cached data for timing_app_data
core           INFO 	Processing timing data...
req            INFO 	Using cached data for car_data
req            INFO 	Using cached data for position_data
req            INFO 	Using cached data for weather_data
req            INFO 	Using cached data for race_control_messages
core           INFO 	Finished loading data for 20 drivers: ['1', '14', '10', '11', '55', '44', '4', '23', '81', '31', '18', '27', '40', '77', '22', '20', '63', '24', '16', '2']


In [3]:
laps=session.laps
drivers=laps['Driver'].unique()

### FETCHING TELEMETRY DATA

In [4]:
total_telemetry=[]
for i in drivers:
    drivers_laps=laps.pick_driver(i)
    driver_telemetry=drivers_laps.get_car_data()
    
    driver_telemetry['Driver']=i
    total_telemetry.append(driver_telemetry)
telemetry_data=pd.concat(total_telemetry,ignore_index=True)
telemetry_data


,Date,RPM,Speed,nGear,Throttle,Brake,DRS,Source,Time,SessionTime,Driver
0,2023-08-27 13:03:05.416,10093,0,1,15,False,1,car,0 days 00:00:00.082000,0 days 01:02:05.042000,VER
1,2023-08-27 13:03:05.616,10063,0,1,15,False,1,car,0 days 00:00:00.282000,0 days 01:02:05.242000,VER
2,2023-08-27 13:03:05.897,8517,1,1,15,False,1,car,0 days 00:00:00.563000,0 days 01:02:05.523000,VER
3,2023-08-27 13:03:06.097,7117,11,1,15,False,1,car,0 days 00:00:00.763000,0 days 01:02:05.723000,VER
4,2023-08-27 13:03:06.497,4317,24,1,16,False,1,car,0 days 00:00:01.163000,0 days 01:02:06.123000,VER
...,...,...,...,...,...,...,...,...,...,...,...
601328,2023-08-27 15:27:25.367,11670,294,7,99,False,1,car,0 days 02:24:20.033000,0 days 03:26:24.993000,PIA
601329,2023-08-27 15:27:25.527,11699,295,7,99,False,1,car,0 days 02:24:20.193000,0 days 03:26:25.153000,PIA
601330,2023-08-27 15:27:25.687,11742,295,7,99,False,1,car,0 days 02:24:20.353000,0 days 03:26:25.313000,PIA
601331,2023-08-27 15:27:26.047,11758,296,7,99,False,1,car,0 days 02:24:20.713000,0 days 03:26:25.673000,PIA


### FETCHING WEATHER,TYRE,LAP AND POSITION DATA 

In [65]:
weather_data=pd.DataFrame(fap.weather_data(session.api_path))
tyre_data = laps[['Driver', 'LapNumber', 'Compound', 'FreshTyre','TyreLife']]
lap_data=laps[['Time','Driver','DriverNumber','LapNumber','LapTime','Sector1Time','Sector2Time','Sector3Time','IsAccurate']]
position_data=laps[['Driver','LapNumber','Position']]
laps

req            INFO 	Using cached data for weather_data


,Time,Driver,DriverNumber,LapTime,LapNumber,Stint,PitOutTime,PitInTime,Sector1Time,Sector2Time,...,FreshTyre,Team,LapStartTime,LapStartDate,TrackStatus,Position,Deleted,DeletedReason,FastF1Generated,IsAccurate
0,0 days 01:03:36.820000,VER,1,0 days 00:01:31.585000,1.0,1.0,NaT,NaT,NaT,0 days 00:00:28.929000,...,True,Red Bull Racing,0 days 01:02:04.960000,2023-08-27 13:03:05.334,1,1.0,False,,False,False
1,0 days 01:05:26.792000,VER,1,0 days 00:01:49.972000,2.0,1.0,NaT,0 days 01:05:25.519000,0 days 00:00:36.330000,0 days 00:00:36.685000,...,True,Red Bull Racing,0 days 01:03:36.820000,2023-08-27 13:04:37.194,1,3.0,False,,False,False
2,0 days 01:07:15.450000,VER,1,0 days 00:01:48.658000,3.0,2.0,0 days 01:05:45.581000,NaT,0 days 00:00:49.822000,0 days 00:00:30.905000,...,True,Red Bull Racing,0 days 01:05:26.792000,2023-08-27 13:06:27.166,1,5.0,False,,False,False
3,0 days 01:08:40.701000,VER,1,0 days 00:01:25.251000,4.0,2.0,NaT,NaT,0 days 00:00:29.996000,0 days 00:00:29.479000,...,True,Red Bull Racing,0 days 01:07:15.450000,2023-08-27 13:08:15.824,1,4.0,False,,False,True
4,0 days 01:10:04.162000,VER,1,0 days 00:01:23.461000,5.0,2.0,NaT,NaT,0 days 00:00:28.478000,0 days 00:00:29.064000,...,True,Red Bull Racing,0 days 01:08:40.701000,2023-08-27 13:09:41.075,1,4.0,False,,False,True
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1338,0 days 03:20:55.371000,PIA,81,0 days 00:01:25.004000,68.0,6.0,NaT,NaT,0 days 00:00:29.890000,0 days 00:00:29.060000,...,False,McLaren,0 days 03:19:30.367000,2023-08-27 15:20:30.741,1,9.0,False,,False,True
1339,0 days 03:22:18.282000,PIA,81,0 days 00:01:22.911000,69.0,6.0,NaT,NaT,0 days 00:00:28.834000,0 days 00:00:28.632000,...,False,McLaren,0 days 03:20:55.371000,2023-08-27 15:21:55.745,1,9.0,False,,False,True
1340,0 days 03:23:40.941000,PIA,81,0 days 00:01:22.659000,70.0,6.0,NaT,NaT,0 days 00:00:28.662000,0 days 00:00:28.457000,...,False,McLaren,0 days 03:22:18.282000,2023-08-27 15:23:18.656,1,9.0,False,,False,True
1341,0 days 03:25:03.237000,PIA,81,0 days 00:01:22.296000,71.0,6.0,NaT,NaT,0 days 00:00:28.423000,0 days 00:00:28.419000,...,False,McLaren,0 days 03:23:40.941000,2023-08-27 15:24:41.315,1,9.0,False,,False,True


### FETCHING RACE CONTROL MESSAGES

In [66]:
rcm_data = pd.DataFrame(fap.race_control_messages(session.api_path))
msg=rcm_data[['Time','Status','Message']]

#Filter for Yellow, Red, VSC, SC 
yellow_flags_key=['YELLOW','DOUBLE YELLOW']
red_flags_key=['RED']
vsc_key=['VSC','VIRTUAL SAFETY CAR']
sc_key=['SC','SAFETY CAR']

#Searching using the keyword in REGEX
yellow_flags = msg[msg['Message'].apply(lambda x: any(re.search(r'\b{}\b'.format(keyword), x) for keyword in yellow_flags_key))]
red_flags = msg[msg['Message'].apply(lambda x: any(re.search(r'\b{}\b'.format(keyword), x) for keyword in red_flags_key))]
vsc = msg[msg['Message'].apply(lambda x: any(re.search(r'\b{}\b'.format(keyword), x) for keyword in vsc_key))]
sc = msg[msg['Message'].apply(lambda x: any(re.search(r'\b{}\b'.format(keyword), x) for keyword in sc_key))]


req            INFO 	Using cached data for race_control_messages


### FETCHING LAP DATA (PITS STOPS, DRIVER POSITION ETC.)

In [105]:
lapsdata, streamdata = fap.timing_data(session.api_path)
# pd.set_option('display.max_columns', None)
lapsdata=pd.DataFrame(lapsdata)
lapsdata['DriverNumber'] = lapsdata['Driver']
gapdata=pd.DataFrame(streamdata)
gapdata['DriverNumber'] = gapdata['Driver']
gap_data = gapdata[['Time','DriverNumber','Position','GapToLeader','IntervalToPositionAhead']]
laps_data=lapsdata[['Time','DriverNumber','LapTime','NumberOfPitStops','PitOutTime','PitInTime']]



req            INFO 	Using cached data for _extended_timing_data


### GETTING CORNERS INFO

In [106]:
circuit_info=session.get_circuit_info()
corners_data=circuit_info.corners

In [107]:
laps_data

,Time,DriverNumber,LapTime,NumberOfPitStops,PitOutTime,PitInTime
0,0 days 01:03:36.820000,1,NaT,0,0 days 00:14:44.885000,NaT
1,0 days 01:05:26.792000,1,0 days 00:01:49.972000,0,NaT,0 days 01:05:25.519000
2,0 days 01:07:15.450000,1,0 days 00:01:48.658000,1,0 days 01:05:45.581000,NaT
3,0 days 01:08:40.701000,1,0 days 00:01:25.251000,1,NaT,NaT
4,0 days 01:10:04.162000,1,0 days 00:01:23.461000,1,NaT,NaT
...,...,...,...,...,...,...
1336,0 days 03:21:01.503000,20,0 days 00:01:25.674000,6,NaT,NaT
1337,0 days 03:22:25.595000,20,0 days 00:01:24.092000,6,NaT,NaT
1338,0 days 03:23:48.622000,20,0 days 00:01:23.027000,6,NaT,NaT
1339,0 days 03:25:12.534000,20,0 days 00:01:23.912000,6,NaT,NaT


In [116]:
master_data = lap_data.merge(position_data, on=['Driver', 'LapNumber'], how='left')
master_data = master_data.merge(tyre_data, on=['Driver', 'LapNumber'], how='left')
master_data = master_data.merge(laps_data, on=['DriverNumber','Time','LapTime'], how='left')

In [117]:
master_data

,Time,Driver,DriverNumber,LapNumber,LapTime,Sector1Time,Sector2Time,Sector3Time,IsAccurate,Position,Compound,FreshTyre,TyreLife,NumberOfPitStops,PitOutTime,PitInTime
0,0 days 01:03:36.820000,VER,1,1.0,0 days 00:01:31.585000,NaT,0 days 00:00:28.929000,0 days 00:00:30.478000,False,1.0,SOFT,True,1.0,NaN,NaT,NaT
1,0 days 01:05:26.792000,VER,1,2.0,0 days 00:01:49.972000,0 days 00:00:36.330000,0 days 00:00:36.685000,0 days 00:00:36.957000,False,3.0,SOFT,True,2.0,0.0,NaT,0 days 01:05:25.519000
2,0 days 01:07:15.450000,VER,1,3.0,0 days 00:01:48.658000,0 days 00:00:49.822000,0 days 00:00:30.905000,0 days 00:00:27.931000,False,5.0,INTERMEDIATE,True,1.0,1.0,0 days 01:05:45.581000,NaT
3,0 days 01:08:40.701000,VER,1,4.0,0 days 00:01:25.251000,0 days 00:00:29.996000,0 days 00:00:29.479000,0 days 00:00:25.776000,True,4.0,INTERMEDIATE,True,2.0,1.0,NaT,NaT
4,0 days 01:10:04.162000,VER,1,5.0,0 days 00:01:23.461000,0 days 00:00:28.478000,0 days 00:00:29.064000,0 days 00:00:25.919000,True,4.0,INTERMEDIATE,True,3.0,1.0,NaT,NaT
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1338,0 days 03:20:55.371000,PIA,81,68.0,0 days 00:01:25.004000,0 days 00:00:29.890000,0 days 00:00:29.060000,0 days 00:00:26.054000,True,9.0,INTERMEDIATE,False,9.0,5.0,NaT,NaT
1339,0 days 03:22:18.282000,PIA,81,69.0,0 days 00:01:22.911000,0 days 00:00:28.834000,0 days 00:00:28.632000,0 days 00:00:25.445000,True,9.0,INTERMEDIATE,False,10.0,5.0,NaT,NaT
1340,0 days 03:23:40.941000,PIA,81,70.0,0 days 00:01:22.659000,0 days 00:00:28.662000,0 days 00:00:28.457000,0 days 00:00:25.540000,True,9.0,INTERMEDIATE,False,11.0,5.0,NaT,NaT
1341,0 days 03:25:03.237000,PIA,81,71.0,0 days 00:01:22.296000,0 days 00:00:28.423000,0 days 00:00:28.419000,0 days 00:00:25.454000,True,9.0,INTERMEDIATE,False,12.0,5.0,NaT,NaT


In [123]:
# master_data = master_data.merge(telemetry_data, on=['Driver', 'LapNumber'], how='left')
# master_data = master_data.merge(weather_data, on='Time', how='left')

,Time,AirTemp,Humidity,Pressure,Rainfall,TrackTemp,WindDirection,WindSpeed
0,0 days 00:00:46.338000,17.4,68.0,1007.4,False,25.5,196,2.7
1,0 days 00:01:46.337000,17.4,67.0,1007.5,False,25.5,193,3.2
2,0 days 00:02:46.336000,17.4,66.0,1007.4,False,25.5,177,3.4
3,0 days 00:03:46.335000,17.5,67.0,1007.4,False,25.6,184,2.4
4,0 days 00:04:46.349000,17.5,67.0,1007.5,False,25.6,186,2.9
5,0 days 00:05:46.364000,17.5,66.0,1007.5,False,25.7,186,2.7
6,0 days 00:06:46.347000,17.6,66.0,1007.5,False,25.9,197,3.0
7,0 days 00:07:46.346000,17.5,66.0,1007.4,False,25.9,177,2.9
8,0 days 00:08:46.345000,17.6,67.0,1007.5,False,26.1,220,3.5
9,0 days 00:09:46.344000,17.7,67.0,1007.4,False,26.3,202,3.2
